In [1]:
%autoawait

IPython autoawait is `on`, and set to use `asyncio`


In [2]:
import time
import asyncio
import flask
from flask import request


In [3]:
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8022
WEBHOOK_BASE = ""
ADMIN_URL = "http://data-agent:8021"

In [4]:
    # Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL, connections=True)

In [5]:
def data_messages_handler(payload):
    connection_id = payload["connection_id"]
    asyncio.get_event_loop().create_task(agent_controller.connections.send_message(connection_id, "THIS IS MY RESPONSE - DATA AGENT"))
    print("Handle data messages ", payload, connection_id)


data_message_listener = {
    "handler": data_messages_handler,
    "topic": "basicmessages"
}

In [6]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

<Task pending coro=<AriesAgentController.listen_webhooks() running at /opt/conda/lib/python3.7/site-packages/aries_basic_controller/aries_controller.py:42>>

In [7]:
agent_controller.register_listeners([data_message_listener], defaults=True)

In [8]:
# Create Invitation
invite = await agent_controller.connections.create_invitation(alias="Will")
print(invite)

{'connection_id': 'fe0b1101-649c-4e23-9084-9bc6dd5e6150', 'invitation': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '1c8f447e-271f-43cf-be83-2f647d5ebcb2', 'recipientKeys': ['C5gih4nhTrQUUcrQ4bWxNiz5R6Jbw1WJ8GrcNEyT2cTE'], 'serviceEndpoint': 'http://192.168.65.3:8020', 'label': 'Bob'}, 'invitation_url': 'http://192.168.65.3:8020?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiMWM4ZjQ0N2UtMjcxZi00M2NmLWJlODMtMmY2NDdkNWViY2IyIiwgInJlY2lwaWVudEtleXMiOiBbIkM1Z2loNG5oVHJRVVVjclE0Yld4Tml6NVI2SmJ3MVdKOEdyY05FeVQyY1RFIl0sICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovLzE5Mi4xNjguNjUuMzo4MDIwIiwgImxhYmVsIjogIkJvYiJ9'}


In [9]:
# Print out accepted Invite and Researcher ID
print("Data Owner ID", invite["connection_id"])
data_connection_id = invite["connection_id"]

Data Owner ID fe0b1101-649c-4e23-9084-9bc6dd5e6150


In [10]:
# Accept Request for Invite created
connection = await agent_controller.connections.accept_request(data_connection_id)
print("ACCEPT REQUEST")
print(connection)

True
ACCEPT REQUEST
{'connection_id': 'fe0b1101-649c-4e23-9084-9bc6dd5e6150', 'state': 'response', 'created_at': '2020-06-25 19:24:48.267345Z', 'updated_at': '2020-06-25 19:25:10.421019Z', 'their_label': 'Alice', 'initiator': 'self', 'invitation_key': 'C5gih4nhTrQUUcrQ4bWxNiz5R6Jbw1WJ8GrcNEyT2cTE', 'routing_state': 'none', 'accept': 'manual', 'invitation_mode': 'once', 'my_did': 'EKFusCHGwjMWdpDr3Ti3cC', 'their_did': '5ZZDzLNPUxYtbML35bd89N'}


In [11]:
trust_ping = await agent_controller.connections.trust_ping(data_connection_id, "hello")
print(trust_ping)

{'thread_id': '468a0a64-1904-455c-b386-35c0213cbac5'}


In [12]:
connection = await agent_controller.connections.get_connection(data_connection_id)
print(connection)

Handle data messages  {'connection_id': 'c0ee2610-5c2c-4270-b1eb-806b0379b1a3', 'message_id': '1ee887f8-5c82-45ad-aa52-28d77020eaee', 'content': 'hello from researcher world!', 'state': 'received'} c0ee2610-5c2c-4270-b1eb-806b0379b1a3
{'updated_at': '2020-06-25 19:10:14.347132Z', 'their_label': 'Alice', 'invitation_key': '5SR7ijUQw6PcEsy1BSYPw9j2iuPiuarRfYEFLCun2r15', 'routing_state': 'none', 'my_did': 'HN8sVP5yBHckzo6XNoXe7Z', 'connection_id': 'c0ee2610-5c2c-4270-b1eb-806b0379b1a3', 'their_did': 'MLffN4Mf7Zxniuo9oPmsgn', 'accept': 'manual', 'invitation_mode': 'once', 'initiator': 'self', 'state': 'active', 'created_at': '2020-06-25 19:09:42.822079Z'}


In [13]:
message = await agent_controller.connections.send_message(data_connection_id,"hello from data owner world! WOOPWOOP")
print("BASIC MESSAGE - DATA -> RESEARCH")
print(message)

BASIC MESSAGE - DATA -> RESEARCH
{}


In [16]:
import torch
import syft as sy
from syft.serde import deserialize
from syft.serde import serialize
from syft.workers.base import BaseWorker


class AriesWorker(BaseWorker):
    def _send_msg(self, message: bin, location: BaseWorker) -> bin:
      return self._recv_msg(message)

    def _recv_msg(self, message: bin) -> bin:
      result = deserialize(self.recv_msg(message))
      print(result)
      message = asyncio.get_event_loop().create_task(agent_controller.connections.send_message(data_connection_id, result ))
      print(message)
      return result

In [18]:
hook = sy.TorchHook(torch)
bob = AriesWorker(hook, id="dave")

In [19]:
x = torch.tensor([1,2,3,4,5])




In [20]:


x_ptr = x.send(bob)



None
<Task pending coro=<ConnectionsController.send_message() running at /opt/conda/lib/python3.7/site-packages/aries_basic_controller/connections_controller.py:94>>


TypeError: 'NoneType' object is not subscriptable

In [24]:
print(x_ptr)

(Wrapper)>[PointerTensor | me:69779731990 -> bob:44629951283]
